In [9]:
import torch
from torch.utils.data import DataLoader
from torch.nn import BCELoss
import torch.nn as nn


from importlib import reload

import vitmbt
import mbt
import ablation
import helpers
reload(mbt)
reload(ablation)
reload(helpers)
reload(vitmbt)
from helpers import ClassifierMetrics, display_tensor_as_rgb
# from vitmbt import ViTMBT, train, val
from vitmbt import ViTAudio, train_audio as train, val_audio as val
from data import EmoDataset, new_collate_fn, load_data
import warnings
warnings.filterwarnings('ignore')

from constants import *

In [6]:
DEVICE = "cuda"
PRETRAINED_CHKPT = "./pretrained_models/L_16-i21k-300ep-lr_0.001-aug_medium1-wd_0.1-do_0.1-sd_0.1--imagenet2012-steps_20k-lr_0.01-res_224.npz"
EPOCHS = 200
lr = 0.00001
BATCH_SZ = 2
LABELS = 8
SPLIT = [.9, 0.05, 0.05]
vmbt = ViTAudio(1024, num_class=LABELS, bottle_layer=20, freeze_first=18, num_layers=24)
vmbt = nn.DataParallel(vmbt).cuda()


train_dl, val_dl, test_dl  = load_data(f"{DATA_DIR}/Labels/all_pruned.csv", 
                                       batch_sz=BATCH_SZ,
                                       train_val_test_split=SPLIT)

optimizer = torch.optim.AdamW(vmbt.parameters(), betas=(0.9, 0.999), lr=lr, weight_decay=1.0 / BATCH_SZ)
# optimizer = torch.optim.SGD(vmbt.parameters(), lr=lr, momentum=momentum, weight_decay=1/BATCH_SZ)

loss_func = BCELoss()
train_cls = ClassifierMetrics(task='multilabel', n_labels=LABELS, device=DEVICE)
val_cls = ClassifierMetrics(task='multilabel', n_labels=LABELS, device=DEVICE)

Loading audio layers...
torch.Size([1, 1, 1024]) torch.Size([1, 184, 1024])
Loaded successfully in 4.9s
Reset trainable parameters of layer = 18.norm1
Reset trainable parameters of layer = 18.attn.qkv
Reset trainable parameters of layer = 18.attn.proj
Reset trainable parameters of layer = 18.norm2
Reset trainable parameters of layer = 18.mlp.fc1
Reset trainable parameters of layer = 18.mlp.fc2
Reset trainable parameters of layer = 19.norm1
Reset trainable parameters of layer = 19.attn.qkv
Reset trainable parameters of layer = 19.attn.proj
Reset trainable parameters of layer = 19.norm2
Reset trainable parameters of layer = 19.mlp.fc1
Reset trainable parameters of layer = 19.mlp.fc2


In [11]:
for epoch in range(EPOCHS):
    train_metrics = train(vmbt, train_dl, optimizer, loss_fn=loss_func, cls_metrics=train_cls)
    train_loss, train_f1, train_r, train_p, train_acc = train_metrics
    val_metrics = val(vmbt, val_dl, loss_fn=loss_func, cls_metrics=val_cls)
    val_loss, val_f1, val_r, val_p, val_acc = val_metrics

    print(
        (f"Epoch {epoch + 1}: train_loss {train_loss:.5f}, val_loss {val_loss:.5f}\n"
            f"                   train_precision {train_p}, val_precision {val_p}\n"
            f"                   train_recall {train_r}, val_recall {val_r}\n"
            f"                   train_f1 {train_f1}, val_f1 {val_f1}"
            )
        )
    

  0%|          | 0/485 [00:00<?, ?it/s]

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.